In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import sklearn.preprocessing
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dataset = pd.read_csv('../data/all_data.csv')

In [3]:
# convert encoding
le = sklearn.preprocessing.LabelEncoder()
dataset['res_max_type'] = le.fit_transform(dataset['res_max_type'])

In [4]:
def splitToTrainTest(dataframe, trainPercent):
    recordNum = dataframe.shape[0]
    shuffledIndex = np.random.permutation(np.arange(recordNum))
    trainSetNum = int(recordNum * trainPercent)
    trainDf, testDf = dataframe.iloc[shuffledIndex[:trainSetNum]], dataframe.iloc[shuffledIndex[trainSetNum:]]
    return trainDf, testDf
train_df, test_df = splitToTrainTest(dataset, 0.7)

In [5]:
# ['count', 'type', 'avg_house_rent_price', 'avg_house_sell_price', 'lat', 'lon']

clf = DecisionTreeClassifier(criterion='entropy')
y_labels = ['LEVEL_FELONY', 'LEVEL_MISDEMEANOR', 'LEVEL_VIOLATION']
irrelevant_labels = ['date', 'time'] + y_labels
clf.fit(train_df.drop(irrelevant_labels, axis='columns'), train_df[y_labels])

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
new_df = pd.DataFrame({'importance':clf.feature_importances_, 'feature':train_df.drop(irrelevant_labels, axis='columns').columns.values})
new_df.sort_values('importance', axis = 0, inplace = True)
fig, ax = plt.subplots()
ax.bar(np.arange(len(new_df['importance'].values)), new_df['importance'].values, width = 0.2, color='b')
ax.set_xticks(np.arange(len(new_df['importance'].values)))
ax.set_xticklabels(new_df['feature'].values, rotation=-45)
plt.title('Feature Importance using Decision Tree model')
ax.set_ylabel('Normalized Gini Importance')

NameError: name 'plt' is not defined

In [ ]:
train_df[[col_name for col_name in train_df.columns if col_name not in irrelevant_labels] + y_labels].corr()

In [ ]:
train_predict = clf.predict(train_df.drop(irrelevant_labels, axis='columns'))
test_predict = clf.predict(test_df.drop(irrelevant_labels, axis='columns'))
train_accuracy = metrics.accuracy_score(train_predict, train_df[y_labels])
test_accuracy = metrics.accuracy_score(test_predict, test_df[y_labels])

print('Train accuracy = {}\nTest accuracy = {}'.format(train_accuracy, test_accuracy))